## Calculate Days of Sale at Plant Material Combination.

Author: Syamanthaka B

Created: May 2019

----------------------------------------------------------------------------------------------------------

* Input 1 is inventory master data with plant, material and closing stock.
* Input 2 is price per unit for a plant material combination

The code below wrangles these data sets to get a 3 month forward looking Days of Sale or DOS.
DOS is calculated as total cost of inventory / number of days. 

Total cost of inventory is taken over forward looking 3 months, eg. for the month of Jan, forward looking 3 months would include Feb, Mar and Apr. Similar approach is taken for number of days. 

Note: Currently the calculating the total cost over upcoming 3 months takes a bit of computing time. WIP on time optimizations

#### Necessary imports

In [85]:
import pandas as pd
from datetime import datetime as dt
import time
import numpy as np

#### Open input 1 and clean up

In [2]:
inventory_raw = pd.read_excel("INV_BEST - Backup.xlsx", sheet_name="Sheet1")
inventory_raw.rename(columns=lambda x: x.strip(), inplace=True)
inventory_raw = inventory_raw[['Plnt', 'Material', 'From Date', 'Opening Stock', 'Total Receipt Qties', 
                               'Total Issue Quantities', 'Closing Stock']]
inventory_raw = inventory_raw[~inventory_raw['Plnt'].isnull()] ## Some extra cleaning found coincidentally
inventory_raw.dropna()
inventory_raw['Material'] = inventory_raw['Material'].str.strip()
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock
0,NL92,032210700233,2018-08-01,27.000,8.0,-5.00,30.000
1,NL92,062236202002,2018-08-01,91.000,17.0,-12.00,96.000
2,NL92,0722237002,2018-08-01,0.000,2.0,-2.00,0.000
3,NL92,082201115003,2018-08-01,127.440,0.0,-0.10,127.340
4,NL92,082201165013,2018-08-01,148.675,0.0,-0.08,148.595


#### Add new column for month

In [3]:
inventory_raw['Month'] = pd.DatetimeIndex(inventory_raw['From Date']).month
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month
0,NL92,032210700233,2018-08-01,27.000,8.0,-5.00,30.000,8
1,NL92,062236202002,2018-08-01,91.000,17.0,-12.00,96.000,8
2,NL92,0722237002,2018-08-01,0.000,2.0,-2.00,0.000,8
3,NL92,082201115003,2018-08-01,127.440,0.0,-0.10,127.340,8
4,NL92,082201165013,2018-08-01,148.675,0.0,-0.08,148.595,8


In [4]:
## Some cosmetic edits
for i in range(3,7):
    inventory_raw.iloc[:,i] = inventory_raw.iloc[:,i].apply(lambda x: round(x))
inventory_raw.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month
0,NL92,032210700233,2018-08-01,27,8,-5,30,8
1,NL92,062236202002,2018-08-01,91,17,-12,96,8
2,NL92,0722237002,2018-08-01,0,2,-2,0,8
3,NL92,082201115003,2018-08-01,127,0,0,127,8
4,NL92,082201165013,2018-08-01,149,0,0,149,8


#### Open input 2 and process

In [94]:
std_price_raw = pd.read_excel("STD_PRICE.xlsx", sheet_name="Sheet1")
std_price_raw.rename(columns=lambda x: x.strip(), inplace=True)
std_price_raw.head()

,Plnt,Material,Material Description,KEY,Mat_Type,MRPpr,ProcType,STD_PRICE
0,NL92,0322-107-00233,WIRE BRAID 14AWG CU/SN,0322-107-00233:NL92,HAWA,BHWA,F,6.24
1,NL92,0622-362-02002,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,0622-362-02002:NL92,FERT,BHWA,F,75.32
2,NL92,0722-237-002,2LG B-CAB. FD12 MID,0722-237-002:NL92,ZMAT,BZMT,E,999999.00
3,NL92,0822-011-15003,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",0822-011-15003:NL92,HAWA,BHWA,F,4.21
4,NL92,0822-011-65013,SH-SLEEVE RNF-3000-12/4-X-SP,0822-011-65013:NL92,HAWA,PMC1,F,3.92


In [95]:
## Clean up
std_price_raw['Material'] = std_price_raw['Material'].str.replace('-', '')
std_price_raw.head()

,Plnt,Material,Material Description,KEY,Mat_Type,MRPpr,ProcType,STD_PRICE
0,NL92,032210700233,WIRE BRAID 14AWG CU/SN,0322-107-00233:NL92,HAWA,BHWA,F,6.24
1,NL92,062236202002,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,0622-362-02002:NL92,FERT,BHWA,F,75.32
2,NL92,0722237002,2LG B-CAB. FD12 MID,0722-237-002:NL92,ZMAT,BZMT,E,999999.00
3,NL92,082201115003,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",0822-011-15003:NL92,HAWA,BHWA,F,4.21
4,NL92,082201165013,SH-SLEEVE RNF-3000-12/4-X-SP,0822-011-65013:NL92,HAWA,PMC1,F,3.92


In [7]:
## Selecting required columns
std_price_raw = std_price_raw.drop_duplicates()
std_price_tmp = std_price_raw[['Plnt', 'Material', 'STD_PRICE']]
std_price_tmp.head()

,Plnt,Material,STD_PRICE
0,NL92,032210700233,6.24
1,NL92,062236202002,75.32
2,NL92,0722237002,999999.00
3,NL92,082201115003,4.21
4,NL92,082201165013,3.92


#### Merge the two inputs

In [8]:
inventory_df = pd.merge(inventory_raw, std_price_tmp, how="left", on=['Plnt', 'Material'])
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92


#### Calculate total cost of closing stock

In [9]:
inventory_df['Total_cost'] = inventory_df['Closing Stock'] * inventory_df['STD_PRICE']
inventory_df.head()

,Plnt,Material,From Date,Opening Stock,Total Receipt Qties,Total Issue Quantities,Closing Stock,Month,STD_PRICE,Total_cost
0,NL92,032210700233,2018-08-01,27,8,-5,30,8,6.24,187.20
1,NL92,062236202002,2018-08-01,91,17,-12,96,8,75.32,7230.72
2,NL92,0722237002,2018-08-01,0,2,-2,0,8,999999.00,0.00
3,NL92,082201115003,2018-08-01,127,0,0,127,8,4.21,534.67
4,NL92,082201165013,2018-08-01,149,0,0,149,8,3.92,584.08


#### Aggregate for plant material and month

In [10]:
inventory_pm = inventory_df.groupby(['Plnt', 'Material', 'Month'])['Total_cost'].agg('sum').reset_index()
inventory_pm.head()

,Plnt,Material,Month,Total_cost
0,NL92,012204310004,5,0.00
1,NL92,012204310004,7,738.40
2,NL92,032210700233,3,199.68
3,NL92,032210700233,4,193.44
4,NL92,032210700233,5,199.68


In [18]:
print(inventory_pm.shape)

(68119, 4)


In [56]:
temp = inventory_pm.iloc[:100]
temp['Agg_sum'] = ''
#temp

C:\Users\320054667\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Function for forward looking cost of consumption. 
Needs performance optimization

In [59]:
##TODO needs further optimiation
def calc_3magg(row):
    month_no = row['Month']
    plant = row['Plnt']
    mat = row['Material']
    
    np3 = [month_no + 1, month_no + 2, month_no + 3]

    m3sum = inventory_pm.loc[(inventory_pm['Plnt'] == plant) & (inventory_pm['Material'] == mat) & (inventory_pm['Month'].isin(np3))]['Total_cost'].agg('sum')
   
    return (m3sum)
    
start = time.time()
#temp['Agg_sum'] = temp.apply(calc_3magg, axis=1)
inventory_pm['Agg_sum'] = inventory_pm.apply(calc_3magg, axis=1)
end = time.time()
print(end - start)
inventory_pm.head()
#temp.head()

902.8549842834473


,Plnt,Material,Month,Total_cost,Agg_sum
0,NL92,012204310004,5,0.00,738.40
1,NL92,012204310004,7,738.40,0.00
2,NL92,032210700233,3,199.68,580.32
3,NL92,032210700233,4,193.44,555.36
4,NL92,032210700233,5,199.68,542.88


#### Creating a month to no. of days reference df

In [62]:
month_data = [[1,31], [2,28], [3,31], [4,30], [5,31], [6,30], [7,31], [8,31], [9,30], [10,31], [11,30], [12,31]]
month_day_df = pd.DataFrame(month_data, columns = ['Month', 'NDays'])
month_day_df.head()

,Month,NDays
0,1,31
1,2,28
2,3,31
3,4,30
4,5,31


In [79]:
for idx, row in month_day_df.iterrows():
    np3m = [row.Month + 1, row.Month + 2, row.Month + 3]
    n3days = month_day_df.loc[month_day_df['Month'].isin(np3m)]['NDays'].agg('sum')
    month_day_df.loc[idx,'N3days'] = n3days

lq = [10,11,12]
lastq = month_day_df['Month'].isin(lq)
month_day_df['N3days'][lastq] = 0
month_day_df

,Month,NDays,N3days
0,1,31,89
1,2,28,92
2,3,31,91
3,4,30,92
4,5,31,92
5,6,30,92
6,7,31,92
7,8,31,91
8,9,30,92
9,10,31,0


#### Adding the month and no. of days forward looking to the inventory df

In [80]:
inventory_pm = pd.merge(inventory_pm, month_day_df, how="left", on=['Month'])
inventory_pm.head()

,Plnt,Material,Month,Total_cost,Agg_sum,NDays,N3days
0,NL92,012204310004,5,0.00,738.40,31,92
1,NL92,012204310004,7,738.40,0.00,31,92
2,NL92,032210700233,3,199.68,580.32,31,91
3,NL92,032210700233,4,193.44,555.36,30,92
4,NL92,032210700233,5,199.68,542.88,31,92


#### Calculating DOS

In [81]:
inventory_pm['DOS'] = inventory_pm['Agg_sum']/inventory_pm['N3days']
inventory_pm.head()

,Plnt,Material,Month,Total_cost,Agg_sum,NDays,N3days,DOS
0,NL92,012204310004,5,0.00,738.40,31,92,8.026087
1,NL92,012204310004,7,738.40,0.00,31,92,0.000000
2,NL92,032210700233,3,199.68,580.32,31,91,6.377143
3,NL92,032210700233,4,193.44,555.36,30,92,6.036522
4,NL92,032210700233,5,199.68,542.88,31,92,5.900870


#### Final cleaning

In [87]:
inventory_pm['DOS'].replace([np.inf], 100000000, inplace=True)
inventory_pm['DOS'].replace([np.nan], 0, inplace=True)
inventory_pm[inventory_pm['N3days'] == 0]

,Plnt,Material,Month,Total_cost,Agg_sum,NDays,N3days,DOS
9,NL92,032210700233,10,243.36,436.80,31,0,100000000.0
10,NL92,032210700233,11,224.64,212.16,30,0,100000000.0
11,NL92,032210700233,12,212.16,0.00,31,0,0.0
19,NL92,062236202002,10,5799.64,12955.04,31,0,100000000.0
20,NL92,062236202002,11,6703.48,6251.56,30,0,100000000.0
21,NL92,062236202002,12,6251.56,0.00,31,0,0.0
23,NL92,0781439102,11,0.00,0.00,30,0,0.0
28,NL92,082201115003,10,534.67,534.67,31,0,100000000.0
29,NL92,082201115003,11,534.67,0.00,30,0,0.0
40,NL92,082201180008,11,302.00,302.00,30,0,100000000.0


In [89]:
inventory_pm['DOS'] = inventory_pm['DOS'].apply(lambda x: round(x))

In [92]:
inventory_pm.rename(columns={'Agg_sum':'ADC'}, inplace=True)
inventory_pm.head()

,Plnt,Material,Month,Total_cost,ADC,NDays,N3days,DOS
0,NL92,012204310004,5,0.00,738.40,31,92,8
1,NL92,012204310004,7,738.40,0.00,31,92,0
2,NL92,032210700233,3,199.68,580.32,31,91,6
3,NL92,032210700233,4,193.44,555.36,30,92,6
4,NL92,032210700233,5,199.68,542.88,31,92,6


#### Add a flag to see if there is demand or not

In [93]:
inventory_pm['Demand_flag'] = np.where(inventory_pm['ADC'] == 0, 'X', '')
inventory_pm.head()

,Plnt,Material,Month,Total_cost,ADC,NDays,N3days,DOS,Demand_flag
0,NL92,012204310004,5,0.00,738.40,31,92,8,
1,NL92,012204310004,7,738.40,0.00,31,92,0,X
2,NL92,032210700233,3,199.68,580.32,31,91,6,
3,NL92,032210700233,4,193.44,555.36,30,92,6,
4,NL92,032210700233,5,199.68,542.88,31,92,6,


In [97]:
std_price_raw = std_price_raw.drop_duplicates()
std_price_raw = std_price_raw[['Plnt', 'Material', 'Material Description', 'Mat_Type', 'MRPpr', 'ProcType']]
std_price_raw.head()

,Plnt,Material,Material Description,Mat_Type,MRPpr,ProcType
0,NL92,032210700233,WIRE BRAID 14AWG CU/SN,HAWA,BHWA,F
1,NL92,062236202002,SOLDERWIRE 97SC 309 5C 1.2mm 0.5kg,FERT,BHWA,F
2,NL92,0722237002,2LG B-CAB. FD12 MID,ZMAT,BZMT,E
3,NL92,082201115003,"HEAT SHRINK, POLYOLEFIN, 1/8"", BK",HAWA,BHWA,F
4,NL92,082201165013,SH-SLEEVE RNF-3000-12/4-X-SP,HAWA,PMC1,F


In [99]:
print(inventory_pm.shape)

(68119, 9)


In [100]:
inventory_pm = pd.merge(inventory_pm, std_price_raw, how="left", on=['Plnt', 'Material'])
print(inventory_pm.shape)
inventory_pm.head()

(68681, 13)


,Plnt,Material,Month,Total_cost,ADC,NDays,N3days,DOS,Demand_flag,Material Description,Mat_Type,MRPpr,ProcType
0,NL92,012204310004,5,0.00,738.40,31,92,8,,"OMSNOERINGSBAND 16X0,5 (pallet a 700 kg)",HAWA,PMC1,F
1,NL92,012204310004,7,738.40,0.00,31,92,0,X,"OMSNOERINGSBAND 16X0,5 (pallet a 700 kg)",HAWA,PMC1,F
2,NL92,032210700233,3,199.68,580.32,31,91,6,,WIRE BRAID 14AWG CU/SN,HAWA,BHWA,F
3,NL92,032210700233,4,193.44,555.36,30,92,6,,WIRE BRAID 14AWG CU/SN,HAWA,BHWA,F
4,NL92,032210700233,5,199.68,542.88,31,92,6,,WIRE BRAID 14AWG CU/SN,HAWA,BHWA,F


In [101]:
## Save file
path = r"C:\Users\\DOS\DOS_calc.xlsx"
writer = pd.ExcelWriter(path, engine='xlsxwriter')
inventory_pm.to_excel(writer, sheet_name='Sheet1')
writer.save()
writer.close()